In [1]:
# BadukPop Analytics Dashboard

# setup
import time, os, calendar, sys, datetime
import envkey
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

engine = create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/tr_upcache', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1640217600
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_dau_sql(game,start_time,end_time,frame_platform=None):
    frame_platform_sql = ''
    if frame_platform:
        frame_platform_sql = "AND acq.frame_platform = '%s'" % frame_platform
    dau_sql = """
    SELECT master.day AS `day`,
    (SELECT ROUND(SUM(dau)/3)
    FROM %s_upcache.%s_sessions_daily_summary AS acq
    WHERE acq.day >= master.day - 3*86400
    AND acq.day < master.day
    %s
    AND acq.country_tier IN ('1','2')) AS `dau`
    FROM skynet.bh_daily_summary master
    WHERE master.day >= %d - 90*86400
    AND master.day < %d
    GROUP BY master.day;
    """ % (game, game, frame_platform_sql,start_time,end_time)
    return dau_sql

In [2]:
time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
dau = {}

for game in ['tr','dv','mf2','bfm','mf','fs']:
    if game in game_frames:
        for frame in game_frames[game]:
            dau_sql = get_dau_sql(game,start_time,end_time,frame)
            dau_data = pd.read_sql(dau_sql, engine)
            if 'day' not in dau:
                dau['day'] = []
                for day in dau_data['day']:
                    dau['day'].append(day)
            key = game.upper() + ' (' +  frame.upper() + ')'
            dau[key] = []
            for day in dau_data['dau']:
                dau[key].append(day)
            if 'total' not in dau:
                dau['total'] = []
                for day in dau_data['dau']:
                    dau['total'].append(day)
            else:
                for i, day in enumerate(dau_data['dau']):
                    dau['total'][i] += day
    else:
        dau_sql = get_dau_sql(game,start_time,end_time)
        dau_data = pd.read_sql(dau_sql, engine)
        if 'day' not in dau:
            dau['day'] = []
            for day in dau_data['day']:
                dau['day'].append(day)
        key = game.upper()
        dau[key] = []
        for day in dau_data['dau']:
            dau[key].append(day)
        if 'total' not in dau:
            dau['total'] = []
            for day in dau_data['dau']:
                dau['total'].append(day)
        else:
            for i, day in enumerate(dau_data['dau']):
                dau['total'][i] += day

In [3]:
print('All Game Tier 1/2 DAU, average of trailing 3 days')
boundaries = get_quarter_time_boundaries(time.time())
boundaries['start'] = boundaries['start'] + 86400*7
for i, day in enumerate(dau['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        print(datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': ' + str(int(dau['total'][i])))

All Game Tier 1/2 DAU, average of trailing 3 days
2021-12-30: 2735
2022-01-06: 2778
2022-01-13: 2776


In [6]:
print('Change since last week (if any)')

boundaries = get_quarter_time_boundaries(time.time())
boundaries['start'] = boundaries['start'] + 86400*7
game_frames = {'tr':['fb','bh','k2'],'fs':['bh']}
for i, day in enumerate(dau['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d') + ': '
        total_changes = 0
        for game in ['tr','dv','mf2','bfm','mf','fs']:
            if game in game_frames:
                for frame in game_frames[game]:
                    key = game.upper() + ' (' +  frame.upper() + ')'
                    this_week = int(dau[key][i])
                    last_week = int(dau[key][i-1])
                    change = this_week - last_week
                    if change != 0:
                        if change > 0:
                            output_line += ' %s: %d,' % (key, change)
                        else:
                        total_changes += 1
            else:
                key = game.upper()
                this_week = dau[key][i]
                last_week = dau[key][i-1]
                change = this_week - last_week
                if change != 0:
                    output_line += ' %s: %d,' % (key, change)
                    total_changes += 1
        if total_changes == 0:
            output_line += 'No Change'
        print(output_line)

Change since last week (if any)
2021-12-30:  TR (FB): 12, TR (BH): -1, TR (K2): 2, MF2: 2, BFM: 3, MF: 6, FS (BH): 4,
2022-01-06:  TR (FB): -6, TR (BH): -1, DV: 3, MF2: 11, BFM: 1, MF: 3, FS (BH): 1,
2022-01-13:  TR (FB): 8, DV: 4, MF2: -1, BFM: -2, MF: 5, FS (BH): 2,
